In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import warnings
warnings.filterwarnings('ignore')


from revolve.grids import MLPParameterGrid
from revolve.architectures import MLPStrategy
from revolve.algorithms import EvolutionaryAlgorithmElitism
from revolve.operators import (
    Operations,
    random_mutation,
    uniform_crossover,
    roulette_wheel_selection,
)

In [2]:
train, test = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)
x_train_full, y_train_full = train
x_test, y_test = test

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.1, random_state=42)

In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [5]:
operations = Operations()
operations.register(roulette_wheel_selection)
operations.register(uniform_crossover, probability=0.5)
operations.register(random_mutation, probability=0.1)

In [6]:
operations.get_operations()

['selection', 'crossover', 'mutation']

In [7]:
mlp_params = MLPParameterGrid(
    input_shape=(x_train.shape[1],),
    regression_target=1,
    regression_activation= "relu",
    hidden_neurons = [0, 16, 32, 64, 128, 258, 512, 1024, 2048],
    activation=['relu', 'swish'],
    dropout=[0.1, 0.2, 0.3, 0.4, 0.5],
    batch_size=[16, 32, 64],
    optimizer=['adam', 'sgd', 'rmsprop'],
    learning_rate=[7.5e-3, 5.0e-3, 2.5e-3, 1.0e-4, 7.5e-4, 5e-4, 2.5e-4],
    l1=[1e-5, 1e-6],
    l2=[1e-5, 1e-6],
)

In [8]:
mlp_params.static_parameters

{'input_shape': (13,), 'regression_target': 1, 'regression_activation': 'relu'}

In [9]:
mlp_params.learnable_parameters

{'hidden_neurons': [0, 16, 32, 64, 128, 258, 512, 1024, 2048],
 'activation': ['relu', 'swish'],
 'l1': [1e-05, 1e-06],
 'l2': [1e-05, 1e-06],
 'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
 'batch_size': [16, 32, 64],
 'optimizer': ['adam', 'sgd', 'rmsprop'],
 'learning_rate': [0.0075, 0.005, 0.0025, 0.0001, 0.00075, 0.0005, 0.00025]}

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2)

mlp = MLPStrategy(
    max_fc = 3,
    squeeze_fc=False,
    epochs=50,
    callback=callback,
    loss='mean_squared_error',
    metric = tfa.metrics.RSquare(),
    parameters = mlp_params
)

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

data = (train_data, valid_data, test_data)

In [12]:
ea = EvolutionaryAlgorithmElitism(
    strategy=mlp,
    pop_size=50,
    elitism_size=1,
    operations=operations,
)

In [13]:
best_chromosome = ea.fit(
    data,
    generations=20,
)

Generations:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Generation 0,                 Best error: 20.854602813720703,                 Best R2 0.7496681213378906


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Generation 1,                 Best error: 18.87354278564453,                 Best R2 0.7735549211502075


...

Generation 17,                 Best error: 16.035694122314453,                 Best R2 0.8174399733543396


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Generation 18,                 Best error: 16.035694122314453,                 Best R2 0.8174399137496948


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Generation 19,                 Best error: 16.035694122314453,                 Best R2 0.8174399137496948


In [14]:
df = ea.results_df()

In [15]:
df.head()

,learnt_parameters,static_parameters,loss,metric,generation
0,"{'fc_0': {'hidden_neurons': 258, 'activation':...","{'input_shape': (13,), 'regression_target': 1,...",26.254490,0.684805,0
1,"{'fc_0': {'hidden_neurons': 32, 'activation': ...","{'input_shape': (13,), 'regression_target': 1,...",31.026688,0.627391,0
2,"{'fc_0': {'hidden_neurons': 2048, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",90.392265,-0.079538,0
3,"{'fc_0': {'hidden_neurons': 1024, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",64.324883,0.227290,0
4,"{'fc_0': {'hidden_neurons': 64, 'activation': ...","{'input_shape': (13,), 'regression_target': 1,...",25.943596,0.688403,0


In [16]:
model = best_chromosome.decode(mlp_params)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 2048)              28672     
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               262272    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 16)                2064      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0     

In [17]:
best_chromosome_row = df[df.loss == df.loss.min()]

In [18]:
dict(best_chromosome_row.learnt_parameters)

{531: {'fc_0': {'hidden_neurons': 512,
   'activation': 'swish',
   'dropout': 0.5,
   'l1': 1e-06,
   'l2': 1e-05},
  'fc_1': {'hidden_neurons': 1024,
   'activation': 'swish',
   'dropout': 0.3,
   'l1': 1e-05,
   'l2': 1e-06},
  'fc_2': {'hidden_neurons': 16,
   'activation': 'relu',
   'dropout': 0.4,
   'l1': 1e-05,
   'l2': 1e-05},
  'batch_size_3': {'batch_size': 32},
  'optimizer_4': {'optimizer': 'sgd'},
  'learning_rate_5': {'learning_rate': 0.0005}},
 600: {'fc_0': {'hidden_neurons': 512,
   'activation': 'swish',
   'dropout': 0.5,
   'l1': 1e-06,
   'l2': 1e-05},
  'fc_1': {'hidden_neurons': 1024,
   'activation': 'swish',
   'dropout': 0.3,
   'l1': 1e-05,
   'l2': 1e-06},
  'fc_2': {'hidden_neurons': 16,
   'activation': 'relu',
   'dropout': 0.4,
   'l1': 1e-05,
   'l2': 1e-05},
  'batch_size_3': {'batch_size': 32},
  'optimizer_4': {'optimizer': 'sgd'},
  'learning_rate_5': {'learning_rate': 0.0005}},
 750: {'fc_0': {'hidden_neurons': 512,
   'activation': 'swish',
   

In [19]:
dict(best_chromosome_row.static_parameters)

{531: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 600: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 750: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 800: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 850: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 900: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 950: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'}}

In [20]:
elite_models = ea.elite_models

In [21]:
elite_models